In [36]:
import pandas as pd
import numpy as np
import astropy.coordinates as coord
from astropy import units as u
import matplotlib.pyplot as plt

In [37]:
data = pd.read_csv("Disk_100pc_old_gal_velocities.csv")

# Positions
r = data['DIST_1000'].to_numpy() * u.pc
RA = data['RA'].to_numpy() * u.deg
DEC = data['DEC'].to_numpy() * u.deg

RA = RA.to(u.rad)
DEC = DEC.to(u.rad)


# Proper motion and radial velocity
pmRA = data['PMRA'].to_numpy() * u.marcsec / u.year
pmDEC = data['PMDEC'].to_numpy() * u.marcsec / u.year
vr = data['RV'].to_numpy() * u.km / u.s

# Change velocities to galactocentric frame
gal_vel = data['(u,v,w)'].to_numpy()

In [48]:
u = np.zeros_like(gal_vel, dtype=np.float32)
v = np.zeros_like(gal_vel, dtype=np.float32)
w = np.zeros_like(gal_vel, dtype=np.float32)

for i,vel in enumerate(gal_vel):
    u[i] = float(vel.split(', ')[0][1:])
    v[i] = float(vel.split(', ')[1])
    w[i] = float(vel.split(', ')[2][:-1])

usun = -np.nanmean(u)
vsun = -np.nanmean(v)
wsun = -np.nanmean(w)

print(usun, vsun, wsun)


9.962144 23.03157 7.8910375


In [38]:

tRA = r * pmRA
tDEC = r * pmDEC

tRA = tRA.to(u.rad * u.km / u.second) 
tDEC = tDEC.to(u.rad * u.km / u.second) 

In [39]:
# Once we have the units we want, we drop the units
r = r.value
RA = RA.value
DEC = DEC.value
vr = vr.value
tRA = tRA.value
tDEC = tDEC.value

In [40]:
xdot = np.zeros_like(vr)
ydot = np.zeros_like(vr)
zdot = np.zeros_like(vr)

for i in range(len(RA)):
    ra = RA[i]
    dec = DEC[i]
    to_cart_matrix =  np.array([[-np.sin(ra), -np.cos(ra) * np.sin(dec), np.cos(ra) * np.cos(dec)],
                               [np.cos(ra) , -np.sin(ra) * np.sin(dec), np.sin(ra) * np.cos(dec)],
                               [0.         , np.cos(dec)              , np.sin(dec)             ]])

    xdot[i], ydot[i], zdot[i] = np.dot(to_cart_matrix, np.array([tRA[i], tDEC[i], vr[i]]))

In [41]:
usun = -np.nanmean(xdot)
vsun = -np.nanmean(ydot)
wsun = -np.nanmean(zdot)

print(usun, vsun, wsun)

3.9866532395204906 -20.509459906610747 15.982327154546583


In [53]:
M = np.cov(np.array([u, v, w]))
Rij = M / (M[0,0] * M[1,1] * M[2,2])

In [55]:
R = np.linalg.det(Rij)

2.680858512637754e-18

In [64]:
def cofactor(A):
    """
    Calculate cofactor matrix of A
    """
    sel_rows = np.ones(A.shape[0],dtype=bool)
    sel_columns = np.ones(A.shape[1],dtype=bool)
    CO = np.zeros_like(A)
    sgn_row = 1
    for row in range(A.shape[0]):
        # Unselect current row
        sel_rows[row] = False
        sgn_col = 1
        for col in range(A.shape[1]):
            # Unselect current column
            sel_columns[col] = False
            # Extract submatrix
            MATij = A[sel_rows][:,sel_columns]
            CO[row,col] = sgn_row*sgn_col*np.linalg.det(MATij)
            # Reselect current column
            sel_columns[col] = True
            sgn_col = -sgn_col
        sel_rows[row] = True
        # Reselect current row
        sgn_row = -sgn_row
    return CO

mat = np.array([[1,2,3],[4,5,6],[7,8,9]])


#cofactor(mat)